In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import cohen_kappa_score
from scipy.stats import mode
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split


import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance
from matplotlib import pyplot
# import shap

In [2]:
os.listdir('.')

['specs.csv',
 '.env',
 'EDA + 1st baseline.ipynb',
 'xgboost-feature-selection-dsbowl.ipynb',
 'sample_submission.csv',
 'data-science-bowl-2019-eda-and-baseline.ipynb',
 'train.csv',
 'train.csv~',
 '.train.csv.swp',
 '.#train.csv',
 'train_labels.csv.zip',
 'test.csv',
 'train_labels.csv',
 '.#test.csv',
 'train.csv.zip',
 'submission.csv',
 '.ipynb_checkpoints',
 '#train.csv#',
 'test.csv.zip']

In [3]:
%%time
train = pd.read_csv('train.csv')
train_labels=pd.read_csv('train_labels.csv',
                         usecols=['installation_id','game_session','accuracy_group'])
test=pd.read_csv('test.csv',usecols=keep_cols)
submission=pd.read_csv('sample_submission.csv')

ParserError: Error tokenizing data. C error: EOF inside string starting at row 3360130

In [4]:
train.head()


NameError: name 'train' is not defined

In [ ]:
train.shape, test.shape


In [ ]:
x = train_labels['accuracy_group'].value_counts()
sns.barplot(x.index,x)

In [8]:
not_req = (set(train.installation_id.unique()) - set(train_labels.installation_id.unique()))

In [9]:
%%time
train_new = ~train['installation_id'].isin(not_req)
train.where(train_new, inplace=True)
train.dropna(inplace=True)
train['event_code']= train.event_code.astype(int)

CPU times: user 15.5 s, sys: 602 ms, total: 16.1 s
Wall time: 16.2 s


In [10]:
def extract_time_features(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['month'] = df['timestamp'].dt.month
    df['hour'] = df['timestamp'].dt.hour
    df['year'] = df['timestamp'].dt.year
    df['dayofweek'] = df['timestamp'].dt.dayofweek
    df['weekofyear'] = df['timestamp'].dt.weekofyear
    return df



In [13]:
## Data preparation
time_features = ['month','hour','year','dayofweek','weekofyear']

def prepare_data(df):
    df = extract_time_features(df)
    df.drop('timestamp', axis=1)
    
    join_one = pd.get_dummies(df[['event_code', 'installation_id', 'game_session']],columns=['event_code']
                             ).groupby([ 'installation_id', 'game_session'], as_index=False,sort=False).agg(sum)
    
    agg={'event_count':sum,'game_time':['sum','mean'],'event_id':'count'}

    join_two = df.drop(time_features, axis=1).groupby(['installation_id', 'game_session'],as_index=False,
                                                      sort=False).agg(agg)
    
    join_two.columns = [' '.join(col).strip() for col in join_two.columns.values]
    
    join_three = df[['installation_id', 'game_session', 'type', 'world', 'title']].groupby(
                    ['installation_id', 'game_session'], as_index=False, sort=False).first()
    
    join_four = df[time_features+['installation_id','game_session']].groupby(['installation_id', 'game_session'],
                                                                            as_index=False,sort=False).agg(mode)[time_features].applymap(lambda x: x.mode[0])
    
    join_one = join_one.join(join_four)
    join_five=(join_one.join(join_two.drop(['installation_id','game_session'],axis=1))). \
                        join(join_three.drop(['installation_id','game_session'],axis=1))
    
    return join_five

In [19]:
%%time
join_testa=prepare_data(test)
cols=join_test.columns.to_list()[2:-3]
join_test[cols]=join_test[cols].astype('int16')

CPU times: user 28 s, sys: 175 ms, total: 28.1 s
Wall time: 28.3 s


In [15]:
%%time
join_test=prepare_data(train)
cols=join_test.columns.to_list()[2:-3]
join_test[cols]=join_test[cols].astype('int16')

CPU times: user 13min 44s, sys: 2.48 s, total: 13min 46s
Wall time: 13min 47s


In [16]:


cols=join_test.columns[2:-12].to_list()
cols.append('event_id count')
cols.append('installation_id')



In [17]:
df=join_test[['event_count sum','game_time mean','game_time sum',
    'installation_id']].groupby('installation_id',as_index=False,sort=False).agg('mean')

df_two=join_test[cols].groupby('installation_id',as_index=False,
                               sort=False).agg('sum').drop('installation_id',axis=1)

df_three=join_test[['title','type','world','installation_id']].groupby('installation_id',
         as_index=False,sort=False).last().drop('installation_id',axis=1)

df_four=join_test[time_features+['installation_id']].groupby('installation_id',as_index=False,sort=False). \
        agg(mode)[time_features].applymap(lambda x : x.mode[0])

In [21]:
final_train=pd.merge(train_labels,join_test,on=['installation_id','game_session'],
                                         how='left').drop(['game_session'],axis=1)


final_test=(df.join(df_two)).join(df_three.join(df_four)).drop('installation_id',axis=1)


In [30]:
df=final_train[['event_count sum','game_time mean','game_time sum','installation_id']]. \
    groupby('installation_id',as_index=False,sort=False).agg('mean')

df_two=final_train.groupby('installation_id',as_index=False,
                                 sort=False).agg('sum').drop('installation_id',axis=1)

df_three=final_train[['accuracy_group','title','type','world','installation_id']]. \
        groupby('installation_id',as_index=False,sort=False). \
        last().drop('installation_id',axis=1)

df_four=join_test[time_features+['installation_id']].groupby('installation_id',as_index=False,sort=False). \
        agg(mode)[time_features].applymap(lambda x : x.mode[0])



final_train=(df.join(df_two)).join(df_three.join(df_four)).drop('installation_id',axis=1)

ValueError: columns overlap but no suffix specified: Index(['event_count sum', 'game_time mean', 'game_time sum'], dtype='object')

In [28]:
final_train[cols]

,event_code_2000,event_code_2010,event_code_2020,event_code_2025,event_code_2030,event_code_2035,event_code_2040,event_code_2050,event_code_2060,event_code_2070,...,event_code_5010,month,hour,year,dayofweek,weekofyear,event_count sum,game_time sum,game_time mean,event_id count
0,1,1,1,1,1,1,0,0,0,0,...,0,8,5,2019,1,32,630,26928,10131,35
1,1,0,1,0,0,0,0,0,0,0,...,0,8,5,2019,1,32,3828,27366,-17765,87
2,1,1,1,1,1,1,0,0,0,0,...,0,8,5,2019,1,32,1176,-10305,17534,48
3,1,1,1,1,1,1,0,0,0,0,...,0,8,20,2019,1,32,903,6334,12633,42
4,1,1,2,0,2,0,0,0,0,0,...,0,8,20,2019,1,32,528,29907,17318,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17685,1,1,2,0,2,0,0,0,0,0,...,0,7,21,2019,0,31,1176,25842,29210,48
17686,1,1,2,0,2,0,0,0,0,0,...,0,8,1,2019,2,32,820,-8034,27651,40
17687,1,1,2,0,2,0,0,0,0,0,...,0,9,2,2019,6,38,1891,-13501,-28155,61
17688,1,0,1,1,0,1,0,0,0,0,...,0,9,1,2019,6,38,4753,24095,-9887,97
